Notebook for Scripts to enhance PPT templates 

Glossary
1) PowerBI Service - Export Bleed
2) PowerBI Desktop - Export Bleed
3) PDF-to-PPT Conversion
4) White Transparency 
   - Dimensions Required
5) PowerPoint Letter Slide Dimensions
6) Adding a PPT Template
   - Need new Template Path
7) Removing Text Boxes
8) Adding Text Boxes based on Mapping
   - Need Mapping
9) Insert Text Boxes with AI Paragraphs
   - Need Mapping and AI Paragraphs (Add)
10) Remove the First Slide of a PowerPoint
11) Remove the Notes section of a PowerPoint
12) Remove Hyperlinks


In [ ]:
# Add Template
from pptx import Presentation
import io
import tkinter as tk
from tkinter import filedialog
import os

def copy_image_to_slide(image_shape, target_presentation, target_slide_layout):
    image_stream = io.BytesIO(image_shape.image.blob)
    image_left, image_top, image_width, image_height = image_shape.left, image_shape.top, image_shape.width, image_shape.height
    target_slide = target_presentation.slides.add_slide(target_slide_layout)
    target_slide.shapes.add_picture(image_stream, image_left, image_top, image_width, image_height)

def process_presentation(source_pptx_path, client_template_path, save_folder):
    source_presentation = Presentation(source_pptx_path)
    client_presentation = Presentation(client_template_path)
    blank_slide_layout = client_presentation.slide_layouts[8]

    for slide in source_presentation.slides:
        for shape in slide.shapes:
            if shape.shape_type == 13:
                copy_image_to_slide(shape, client_presentation, blank_slide_layout)

    base_name = os.path.basename(source_pptx_path)
    new_name = "Updated_" + base_name
    new_pptx_path = os.path.join(save_folder, new_name)
    client_presentation.save(new_pptx_path)
    print(f"New presentation saved as {new_pptx_path}.")

def main():
    root = tk.Tk()
    root.withdraw()

    pptx_files = filedialog.askopenfilenames(
        title="Select PowerPoint files",
        filetypes=[("PowerPoint files", "*.pptx")])
    client_template_path = filedialog.askopenfilename(
        title="Select PowerPoint template",
        filetypes=[("PowerPoint files", "*.pptx")])
    save_folder = filedialog.askdirectory(title="Select save folder")

    if not pptx_files or not client_template_path or not save_folder:
        print("Operation cancelled or incomplete selection.")
        return

    for pptx_path in pptx_files:
        process_presentation(pptx_path, client_template_path, save_folder)

if __name__ == "__main__":
    main()


In [ ]:
# Make images transparent in multiple presentations
from pptx import Presentation
from PIL import Image
import io
import numpy as np
import tkinter as tk
from tkinter import filedialog
import os

def make_white_transparent(img, white_threshold=220, tolerance=10):
    print("Converting image to RGBA...")
    img = img.convert("RGBA")
    datas = np.array(img)

    height, width = datas.shape[:2]
    transparency_start = int(height * 0.90)

    for y in range(transparency_start, height):
        for x in range(width):
            if all(datas[y, x][:3] >= white_threshold - tolerance):
                datas[y, x][3] = 0

    modified_img = Image.fromarray(datas)
    print("Finished making white areas transparent in the bottom 10% of the image.")
    return modified_img

def select_files(title="Select files"):
    root = tk.Tk()
    root.withdraw()
    file_paths = filedialog.askopenfilenames(title=title, filetypes=[('PowerPoint files', '*.pptx')])
    root.destroy()
    return file_paths

def process_presentations(paths):
    for path in paths:
        print(f"Opening presentation from {path}...")
        prs = Presentation(path)
        slide_count = 0
        image_count = 0

        for slide in prs.slides:
            slide_count += 1
            for shape in list(slide.shapes):
                if shape.shape_type == 13:
                    image_count += 1
                    print(f"Processing image {image_count} on slide {slide_count}...")

                    image_stream = io.BytesIO(shape.image.blob)
                    image = Image.open(image_stream)
                    transparent_image = make_white_transparent(image)

                    output_stream = io.BytesIO()
                    transparent_image.save(output_stream, format='PNG')
                    output_stream.seek(0)

                    left, top, width, height = shape.left, shape.top, shape.width, shape.height
                    sp = shape._element
                    sp.getparent().remove(sp)

                    slide.shapes.add_picture(output_stream, left, top, width, height)
                    print(f"Replaced original image with transparent image on slide {slide_count}.")

        output_path = os.path.splitext(path)[0] + "_transparent.pptx"
        prs.save(output_path)
        print(f"Presentation saved to {output_path}")

if __name__ == "__main__":
    selected_files = select_files("Select PowerPoint files to make transparent")
    process_presentations(selected_files)


In [ ]:
# Remove text boxes from PowerPoint presentations
from pptx import Presentation
import tkinter as tk
from tkinter import filedialog
import os

def remove_text_boxes(pptx_path, save_folder):
    prs = Presentation(pptx_path)

    for slide in prs.slides:
        shapes_with_text = [shape for shape in slide.shapes if shape.has_text_frame]

        for shape in shapes_with_text:
            sp = shape._element
            sp.getparent().remove(sp)

    base_name = os.path.basename(pptx_path)
    new_name = "Processed_" + base_name
    new_pptx_path = os.path.join(save_folder, new_name)
    prs.save(new_pptx_path)
    print(f"Processed presentation saved as: {new_pptx_path}")

def main():
    root = tk.Tk()
    root.withdraw()

    pptx_files = filedialog.askopenfilenames(
        title="Select PowerPoint files",
        filetypes=[("PowerPoint files", "*.pptx")])

    save_folder = filedialog.askdirectory(title="Select folder to save modified presentations")

    if not pptx_files or not save_folder:
        print("Operation cancelled or incomplete selection.")
        return

    for pptx_path in pptx_files:
        remove_text_boxes(pptx_path, save_folder)

if __name__ == "__main__":
    main()


In [3]:
# Export Bleed for PowerBI Service
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE
import tkinter as tk
from tkinter import filedialog, ttk
import os
from threading import Thread

# Constants
EMU_PER_INCH = 914400

def crop_image(shape, left, right, top, bottom):
    """Adjusts the cropping of an image."""
    shape.crop_left = left
    shape.crop_right = right
    shape.crop_top = top
    shape.crop_bottom = bottom

def adjust_image_size_and_position(shape, slide_width_emus, slide_height_emus):
    """Sets the image size to match the slide and positions it at the top left corner."""
    shape.width = slide_width_emus
    shape.height = slide_height_emus
    shape.left = 0
    shape.top = 0

def select_files(title="Select files"):
    """Opens a dialog for file selection."""
    root = tk.Tk()
    root.withdraw()
    file_paths = filedialog.askopenfilenames(title=title, filetypes=[('PowerPoint files', '*.pptx')])
    root.destroy()
    return file_paths

def select_save_folder(title="Select a folder"):
    """Opens a dialog for folder selection."""
    root = tk.Tk()
    root.withdraw()
    folder_path = filedialog.askdirectory(title=title)
    root.destroy()
    return folder_path

def process_ppt_files(ppt_paths, save_path, update_ui_callback):
    """Processes each PowerPoint file, adjusting images as specified."""
    for index, ppt_path in enumerate(ppt_paths):
        file_name = os.path.basename(ppt_path)
        prs = Presentation(ppt_path)
        crop_left_value = 0.00
        crop_right_value = 0.00
        crop_top_value = 0.00
        crop_bottom_value = 0.014

        slide_width_emus = int(13.33 * EMU_PER_INCH)
        slide_height_emus = int(7.5 * EMU_PER_INCH)

        for slide in prs.slides:
            for shape in slide.shapes:
                if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                    crop_image(shape, crop_left_value, crop_right_value, crop_top_value, crop_bottom_value)
                    adjust_image_size_and_position(shape, slide_width_emus, slide_height_emus)

        save_path_with_filename = os.path.join(save_path, file_name)
        prs.save(save_path_with_filename)

        update_ui_callback(index + 1)

def update_ui(progress_bar, status_label, current, total_files):
    """Updates the UI with the current progress."""
    progress_bar['value'] = (current / total_files) * 100
    status_label.config(text=f'Processing {current}/{total_files} files...')
    if current == total_files:
        status_label.config(text="Done! Close this window to exit.")
        ttk.Button(progress_window, text="Close", command=progress_window.destroy).pack()

def main():
    """Main function to execute the workflow."""
    ppt_paths = select_files("Select PowerPoint files to adjust")
    total_files = len(ppt_paths)
    if total_files == 0:
        print("No files selected. Exiting.")
        return

    save_path = select_save_folder("Select folder to save adjusted PowerPoints")
    if not save_path:
        print("No save folder selected. Exiting.")
        return

    global progress_window
    progress_window = tk.Tk()
    progress_window.title("Processing PowerPoints")
    ttk.Label(progress_window, text="Progress:").pack()
    progress_bar = ttk.Progressbar(progress_window, orient='horizontal', length=300, mode='determinate')
    progress_bar.pack()
    status_label = ttk.Label(progress_window, text="Starting...")
    status_label.pack()

    processing_thread = Thread(target=process_ppt_files, args=(ppt_paths, save_path, lambda current: update_ui(progress_bar, status_label, current, total_files)))
    processing_thread.start()

    progress_window.mainloop()

if __name__ == "__main__":
    main()


No files selected. Exiting.


In [ ]:
# Export Bleed for PowerBI Desktop

from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE
import tkinter as tk
from tkinter import filedialog, ttk
import os
from threading import Thread

EMU_PER_INCH = 914400

def crop_image(shape, left, right, top, bottom):
    shape.crop_left = left
    shape.crop_right = right
    shape.crop_top = top
    shape.crop_bottom = bottom

def expand_image_to_overflow(shape):
    width_increase_emu = int(0.03 * EMU_PER_INCH)
    height_increase_emu = int(0.03 * EMU_PER_INCH)
    shape.width += width_increase_emu
    shape.height += height_increase_emu
    left_shift_emu = int(0.01 * EMU_PER_INCH)
    top_shift_emu = int(0.01 * EMU_PER_INCH)
    shape.left -= left_shift_emu
    shape.top -= top_shift_emu

def select_files(title="Select files"):
    root = tk.Tk()
    root.withdraw()
    file_paths = filedialog.askopenfilenames(title=title, filetypes=[('PowerPoint files', '*.pptx')])
    root.destroy()
    return file_paths

def select_save_folder(title="Select a folder"):
    root = tk.Tk()
    root.withdraw()
    folder_path = filedialog.askdirectory(title=title)
    root.destroy()
    return folder_path

def process_ppt_files(ppt_paths, save_path, update_ui_callback):
    for index, ppt_path in enumerate(ppt_paths):
        file_name = os.path.basename(ppt_path)
        prs = Presentation(ppt_path)
        crop_left_value = 0.020
        crop_right_value = 0.020
        crop_top_value = 0.035
        crop_bottom_value = 0.035

        for slide in prs.slides:
            for shape in slide.shapes:
                if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                    crop_image(shape, crop_left_value, crop_right_value, crop_top_value, crop_bottom_value)
                    expand_image_to_overflow(shape)

        save_path_with_filename = os.path.join(save_path, file_name)
        prs.save(save_path_with_filename)

        update_ui_callback(index + 1)

def update_ui(progress_bar, status_label, current, total_files):
    progress_bar['value'] = (current / total_files) * 100
    status_label.config(text=f'Processing {current}/{total_files} files...')
    if current == total_files:
        status_label.config(text="Done! Close this window to exit.")
        ttk.Button(progress_window, text="Close", command=progress_window.destroy).pack()

def main():
    ppt_paths = select_files("Select PowerPoint files to adjust")
    total_files = len(ppt_paths)
    save_path = select_save_folder("Select folder to save adjusted PowerPoints")

    global progress_window
    progress_window = tk.Tk()
    progress_window.title("Processing PowerPoints")
    ttk.Label(progress_window, text="Progress:").pack()
    progress_bar = ttk.Progressbar(progress_window, orient='horizontal', length=300, mode='determinate')
    progress_bar.pack()
    status_label = ttk.Label(progress_window, text="Starting...")
    status_label.pack()

    processing_thread = Thread(target=process_ppt_files, args=(ppt_paths, save_path, lambda current: update_ui(progress_bar, status_label, current, total_files)))
    processing_thread.start()

    progress_window.mainloop()

if __name__ == "__main__":
    main()


In [ ]:
# PPT Letter Slides
from pptx import Presentation
from pptx.util import Inches

# Path to your PowerPoint file
pptx_file = r''

# Custom slide dimensions
slide_height = Inches(11.04)  # 11.04 inches
slide_width = Inches(8.5)     # 8.5 inches

# Load the presentation
presentation = Presentation(pptx_file)

# Set slide width and height
presentation.slide_width = slide_width
presentation.slide_height = slide_height

# Iterate over each slide and resize images
for slide in presentation.slides:
    for shape in slide.shapes:
        if shape.shape_type == 13:  # Shape type 13 corresponds to a picture
            # Resize image to fill the slide
            shape.width = slide_width
            shape.height = slide_height
            # Center the image on the slide
            shape.left = 0
            shape.top = 0

# Save the modified presentation
presentation.save(pptx_file)


In [ ]:
# PDF to PPT
from pptx import Presentation
from pptx.util import Inches
import fitz  # PyMuPDF
import io
import os
import tkinter as tk
from tkinter import filedialog, ttk

def select_files(title="Select files", filetypes=[('All files', '*.*')]):
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_paths = filedialog.askopenfilenames(title=title, filetypes=filetypes)
    root.destroy()
    return root.tk.splitlist(file_paths)

def select_file(title="Select a file", filetypes=[('All files', '*.*')]):
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    file_path = filedialog.askopenfilename(title=title, filetypes=filetypes)
    root.destroy()
    return file_path

def select_folder(title="Select a folder"):
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    folder_path = filedialog.askdirectory(title=title)
    root.destroy()
    return folder_path

def update_progress(progress_bar, value):
    progress_bar['value'] = value
    progress_bar.update()

# Select the PDF files and PowerPoint template
paths_to_pdfs = select_files("Select the PDF files", [('PDF files', '*.pdf')])
path_to_template = select_file("Select the PowerPoint template", [('PowerPoint files', '*.pptx')])

# Select the output folder
output_folder = select_folder("Select folder to save the PowerPoint")

# Check if the files were selected
if not paths_to_pdfs or not path_to_template or not output_folder:
    raise Exception("File not selected")

# Define the slide dimensions (16:9 PowerPoint slide dimensions)
slide_height = Inches(7.5)  # inches
slide_width = Inches(13.33)  # inches

# Define the margins for the image
left_margin = (Inches(13.33) - slide_width) / 2  # inches
top_margin = (Inches(7.5) - slide_height) / 2  # inches

# Initialize progress bar window
progress_window = tk.Tk()
progress_window.title("Conversion Progress")
total_pages = sum([len(fitz.open(pdf_path)) for pdf_path in paths_to_pdfs])
progress_bar = ttk.Progressbar(progress_window, orient='horizontal', length=300, mode='determinate', maximum=total_pages)
progress_bar.pack()
progress_window.update()

# Process each PDF
for path_to_pdf in paths_to_pdfs:
    pdf_base_name = os.path.splitext(os.path.basename(path_to_pdf))[0]

    # Load the PowerPoint template for each PDF
    prs = Presentation(path_to_template)

    # Load the PDF
    pdf_document = fitz.open(path_to_pdf)

    # Loop through the pages
    for page_num in range(len(pdf_document)):
        update_progress(progress_bar, page_num)

        # Convert the page to an image
        page = pdf_document.load_page(page_num)
        pix = page.get_pixmap(dpi=700)
        img_data = pix.tobytes("png")
        image_stream = io.BytesIO(img_data)

        # Add a slide for the image
        slide_layout = prs.slide_layouts[8]
        slide = prs.slides.add_slide(slide_layout)

        # Add the image to the slide
        pic = slide.shapes.add_picture(image_stream, left_margin, top_margin, slide_width, slide_height)

    # Save the presentation for the current PDF
    output_file = os.path.join(output_folder, pdf_base_name + ".pptx")
    prs.save(output_file)
    print(f"Presentation for {pdf_base_name} saved at {output_file}")

# Final update to the progress bar
update_progress(progress_bar, total_pages)
progress_window.destroy()


In [ ]:
# Insert Text Box's based on Mapping
import os
from pptx import Presentation
from pptx.util import Pt

# Define paths
ppt_folder = r''
master_ppt_path = r''

# Define the mapping array
# Indexes are 0-based, so Slide 1 is index 0, Slide 2 is index 1, etc.
mapping = {
    0: 2,
    1: 6,
    2: 7,
    3: 8,
    4: 11,
    5: 12,
    6: 13
}

def copy_text_from_master_to_presentation(master_ppt_path, target_ppt_path, mapping):
    master_ppt = Presentation(master_ppt_path)
    target_ppt = Presentation(target_ppt_path)

    for source_index, target_index in mapping.items():
        while len(target_ppt.slides) <= target_index:
            target_ppt.slides.add_slide(target_ppt.slide_layouts[5])

        master_slide = master_ppt.slides[source_index]
        target_slide = target_ppt.slides[target_index]

        for shape in master_slide.shapes:
            if shape.shape_type == 17:
                new_shape = target_slide.shapes.add_textbox(shape.left, shape.top, shape.width, shape.height)
                new_text_frame = new_shape.text_frame
                new_text_frame.clear()

                first_paragraph = True
                for paragraph in shape.text_frame.paragraphs:
                    if first_paragraph:
                        new_paragraph = new_text_frame.paragraphs[0]
                        first_paragraph = False
                    else:
                        new_paragraph = new_text_frame.add_paragraph()

                    new_paragraph.space_before = Pt(0)
                    new_paragraph.space_after = Pt(0)
                    for run in paragraph.runs:
                        new_run = new_paragraph.add_run()
                        new_run.text = run.text
                        new_run.font.bold = run.font.bold
                        new_run.font.italic = run.font.italic
                        new_run.font.underline = run.font.underline
                        new_run.font.size = run.font.size
                        new_run.font.name = run.font.name

    target_ppt.save(target_ppt_path)

def process_presentations(ppt_folder, master_ppt_path, mapping):
    for filename in os.listdir(ppt_folder):
        if filename.endswith(".pptx"):
            target_ppt_path = os.path.join(ppt_folder, filename)
            copy_text_from_master_to_presentation(master_ppt_path, target_ppt_path, mapping)
            print(f"Processed {target_ppt_path}")

if __name__ == "__main__":
    process_presentations(ppt_folder, master_ppt_path, mapping)

print("Text boxes insertion complete.")


In [ ]:
# Remove First Slide of PowerPoint
import os
from pptx import Presentation

def remove_first_slide_from_pptx(file_path):
    prs = Presentation(file_path)

    if len(prs.slides) > 0:
        xml_slides = prs.slides._sldIdLst
        slides = list(xml_slides)
        xml_slides.remove(slides[0])

    prs.save(file_path)

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.pptx'):
            file_path = os.path.join(folder_path, filename)
            remove_first_slide_from_pptx(file_path)
            print(f"Processed {file_path}")

if __name__ == "__main__":
    folder_path = r''
    process_folder(folder_path)

In [1]:
# Remove Notes from PowerPoint
import os
from pptx import Presentation

def remove_notes_from_pptx(file_path):
    prs = Presentation(file_path)

    for slide in prs.slides:
        if slide.has_notes_slide:
            slide.notes_slide.notes_text_frame.clear()

    prs.save(file_path)

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.pptx'):
            file_path = os.path.join(folder_path, filename)
            remove_notes_from_pptx(file_path)
            print(f"Processed {file_path}")

if __name__ == "__main__":
    folder_path = r''
    process_folder(folder_path)

Processed C:\Users\DavidShevchenko\Downloads\ExxonMobil Tier 1 Deck Template.pptx
Processed C:\Users\DavidShevchenko\Downloads\ExxonMobil Tier 2 Deck Template.pptx
Processed C:\Users\DavidShevchenko\Downloads\ExxonMobil Tier 3 Deck Template.pptx


In [ ]:
# Insert Paragraphs into PowerPoint
import os
import pandas as pd
from pptx import Presentation
from pptx.enum.dml import MSO_COLOR_TYPE

# Load the Excel document
excel_path = r''
data = pd.read_excel(excel_path)

# Define the folder containing PowerPoint files and the folder to save the updated PowerPoint files
ppt_folder = r''
output_folder = r''

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

def replace_placeholders(slide, label_dict):
    for shape in slide.shapes:
        if shape.has_text_frame:
            text_frame = shape.text_frame
            text_frame.word_wrap = True
            for paragraph in text_frame.paragraphs:
                runs_text = [run.text for run in paragraph.runs]
                full_text = ''.join(runs_text)
                original_runs = paragraph.runs

                for label, value in label_dict.items():
                    placeholder = f"<{label}>"
                    if placeholder in full_text:
                        full_text = full_text.replace(placeholder, str(value))

                if full_text != ''.join(runs_text):
                    paragraph.clear()
                    new_run = paragraph.add_run()
                    new_run.text = full_text

                    if original_runs:
                        first_run_font = original_runs[0].font
                        new_run.font.name = first_run_font.name
                        new_run.font.bold = first_run_font.bold
                        new_run.font.italic = first_run_font.italic
                        new_run.font.underline = first_run_font.underline
                        new_run.font.size = first_run_font.size
                        if first_run_font.color and first_run_font.color.type == MSO_COLOR_TYPE.RGB:
                            new_run.font.color.rgb = first_run_font.color.rgb

# Collect deck titles from the Excel data
excel_deck_titles = set(data['decktitle'].astype(str))

# Collect all PPT files from the folder
ppt_files = [f for f in os.listdir(ppt_folder) if f.lower().endswith('.pptx')]
ppt_titles = {os.path.splitext(f)[0] for f in ppt_files}

# Loop over each row in the Excel document
for _, row in data.iterrows():
    deck_title = str(row['decktitle'])
    ppt_path = os.path.join(ppt_folder, f"{deck_title}.pptx")

    if os.path.exists(ppt_path):
        presentation = Presentation(ppt_path)
        label_dict = row.to_dict()

        for slide in presentation.slides:
            replace_placeholders(slide, label_dict)

        new_ppt_path = os.path.join(output_folder, f"{deck_title}.pptx")
        presentation.save(new_ppt_path)
    else:
        print(f"PowerPoint file not found for decktitle in Excel: {ppt_path}")

# Identify PPTs that are not referenced in the Excel
unreferenced_ppts = ppt_titles - excel_deck_titles
if unreferenced_ppts:
    print("The following PPT files were found in the folder but not in the Excel document:")
    for ppt in unreferenced_ppts:
        print(ppt)

print("Processing complete.")


In [ ]:
import os
from pptx import Presentation

def remove_hyperlinks(shape):
    # If it's a group shape, recurse into each child shape
    if shape.shape_type == 6:
        for sub_shape in shape.shapes:
            remove_hyperlinks(sub_shape)
    else:
        # For non-group shapes, remove the hyperlink if present
        if hasattr(shape, 'click_action') and shape.click_action and shape.click_action.hyperlink:
            shape.click_action.hyperlink.address = None

def remove_hyperlinks_from_pptx(pptx_path):
    prs = Presentation(pptx_path)
    for slide in prs.slides:
        for shape in slide.shapes:
            remove_hyperlinks(shape)
    prs.save(pptx_path)

def process_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.pptx'):
            pptx_path = os.path.join(folder_path, file_name)
            print(f"Processing {file_name}...")
            remove_hyperlinks_from_pptx(pptx_path)
            print(f"Hyperlinks removed from {file_name}.")

if __name__ == "__main__":
    # Path to your folder containing the PPTX files
    folder_path = r""

    # Process the folder
    process_folder(folder_path)
